# Project 3

<b> Group members: </b>
1.	Dianela Perdomo 
2.	Hendyel Pacheco
3.	Lihe Liu
4.	Pornpamol Pattamanont

## Data:
The data file for this problem set is here:
```bash
/ufrc/zoo6927/share/Class_Files/data/flights.May2017-Apr2018.csv 
```

## Plan:
1.	Two tables we are going to create are 1) delayed flights depart from GNV 2) distance of those flights with airlines. 
2.	The first table has 6 columns: FL_DATE /AIRLINE_ID (this will be the foreign key that we will add) / ORIGIN / ORIGIN_CITY_NAME / DEST / DEP_DEL15. The primary key in the first table is a number.
3.	The second table is ‘Airlines table’ which has AIRLINE_ID / DISTANCE/ORIGIN_CITY_NAME / DEST. The key in this table is our foreign key in first table.
4.	We use ‘join’ function to link between these 2 tables; where to check GNV in table 2.
5.	We will analyze how many delayed flights each airline has, and organize from max to min.


## Solution:

<b> Create empty database: </b>

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import DateTime, Boolean

# Create a sqlite database 
engine = create_engine('sqlite:///flights_pj3.sqlite')

metadata=MetaData()
Airports = Table ('Airports', metadata,
                Column('Code', String, primary_key=True),
                Column('FL_DATE', DateTime),
                Column('AIRLINE_ID', Integer, autoincrement=True),
                Column('ORIGIN', String),
                Column('ORIGIN_CITY_NAME', String),
                Column('DEST', String),
                Column('DEP_DEL15', Boolean),
               )
Airlines = Table ('Airlines', metadata,
                 Column('AIRLINE_ID', Integer, ForeignKey("Airports.AIRLINE_ID")),
                 Column('DISTANCE', Integer),
                 Column('ORIGIN_CITY_NAME', String),
                 Column('DEST', String),
                 )
metadata.create_all(engine)


<b> Put the Flights data into dataframe: </b>

In [ ]:
import csv
flights=open("/ufrc/zoo6927/share/Class_Files/data/flights.May2017-Apr2018.csv")

reader = csv.DictReader(flights)
for Line in reader:

    ins=Airports.insert().values(FL_DATE=Line['FL_DATE'],AIRLINE_ID=Line['AIRLINE_ID'],
                                ORIGIN=Line['ORIGIN'],ORIGIN_CITY_NAME=Line['ORIGIN_CITY_NAME'],DEST=Line['DEST'],
                                DEP_DEL15 = Line['DEP_DEL15'])

result = conn.execute(ins)